#**Experiments with the database from the article "Organism-Specific training improves performance of linear B-Cell epitope prediction"**

---




In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/orgspec-epitope-prediction')  

Mounted at /content/drive/


In [ ]:
import os
for dirname, _, filenames in os.walk('./input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./input/02_holdout.rds
./input/training-Ovolvulus.csv


In [ ]:
import pandas as pd 

# Importing the dataset
train_ovolvulus = pd.read_csv('./input/training-Ovolvulus.csv')

train_ovolvulus

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Info_epitope_id,Info_sourceOrg_id,Info_protein_id,Info_host_id,Info_n_Positive,Info_n_Negative,Info_TSeq_accver,Info_TSeq_taxid,Info_TSeq_orgname,Info_center_pos,Info_window_seq,Class,feat_seq_entropy,feat_C_atoms,feat_H_atoms,feat_N_atoms,feat_O_atoms,feat_S_atoms,feat_molecular_weight,feat_Perc_Tiny,feat_Perc_Small,feat_Perc_Aliphatic,feat_Perc_Aromatic,feat_Perc_NonPolar,feat_Perc_Polar,feat_Perc_Charged,feat_Perc_Basic,feat_Perc_Acidic,feat_PP1,feat_PP2,feat_PP3,feat_KF1,feat_KF2,feat_KF3,feat_KF4,feat_KF5,feat_KF6,feat_KF7,feat_KF8,feat_KF9,...,feat_Perc_WA,feat_Perc_WC,feat_Perc_WD,feat_Perc_WE,feat_Perc_WF,feat_Perc_WG,feat_Perc_WH,feat_Perc_WI,feat_Perc_WK,feat_Perc_WL,feat_Perc_WM,feat_Perc_WN,feat_Perc_WP,feat_Perc_WQ,feat_Perc_WR,feat_Perc_WS,feat_Perc_WT,feat_Perc_WV,feat_Perc_WW,feat_Perc_WY,feat_Perc_YA,feat_Perc_YC,feat_Perc_YD,feat_Perc_YE,feat_Perc_YF,feat_Perc_YG,feat_Perc_YH,feat_Perc_YI,feat_Perc_YK,feat_Perc_YL,feat_Perc_YM,feat_Perc_YN,feat_Perc_YP,feat_Perc_YQ,feat_Perc_YR,feat_Perc_YS,feat_Perc_YT,feat_Perc_YV,feat_Perc_YW,feat_Perc_YY
0,854007,6282,A0A044QLL7,"9606,9606",1,1,NaN,6282,Onchocerca volvulus,1,LLLLLLLLDVHIESG,1,2.306891,79,165,17,35,0,1913.23,0.133333,0.266667,0.666667,0.066667,0.733333,0.266667,0.200000,0.066667,0.133333,-0.466667,-0.240000,-0.186667,-0.586667,-0.228000,-0.108667,-0.496667,-0.344667,-1.077333,0.125333,-0.198667,0.180667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,854007,6282,A0A044QLL7,"9606,9606",1,1,NaN,6282,Onchocerca volvulus,2,LLLLLLLDVHIESGE,1,2.463458,78,161,17,37,0,1929.19,0.133333,0.266667,0.600000,0.066667,0.666667,0.333333,0.266667,0.066667,0.200000,-0.344000,-0.278000,-0.236667,-0.614000,-0.215333,-0.200000,-0.345333,-0.395333,-0.914000,0.064000,-0.122667,0.127333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,854007,6282,A0A044QLL7,"9606,9606",1,1,NaN,6282,Onchocerca volvulus,3,LLLLLLDVHIESGEV,1,2.606239,77,159,17,37,0,1915.17,0.133333,0.333333,0.600000,0.066667,0.666667,0.333333,0.266667,0.066667,0.200000,-0.350667,-0.308667,-0.230000,-0.594000,-0.262667,-0.048000,-0.298667,-0.325333,-0.831333,-0.071333,-0.068000,0.066667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,854007,6282,A0A044QLL7,"9606,9606",1,1,NaN,6282,Onchocerca volvulus,4,LLLLLDVHIESGEVC,1,2.866248,74,153,17,37,1,1905.16,0.200000,0.400000,0.533333,0.066667,0.666667,0.333333,0.266667,0.066667,0.200000,-0.327333,-0.342000,-0.201333,-0.516667,-0.322000,-0.002000,-0.295333,-0.336000,-0.534000,-0.034000,-0.063333,0.112000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,854007,6282,A0A044QLL7,"9606,9606",1,1,NaN,6282,Onchocerca volvulus,5,LLLLDVHIESGEVCA,1,3.106891,71,147,17,37,1,1863.08,0.266667,0.466667,0.533333,0.066667,0.666667,0.333333,0.266667,0.066667,0.200000,-0.331333,-0.394667,-0.164667,-0.551333,-0.433333,-0.050667,-0.240000,-0.361333,-0.449333,-0.111333,-0.018000,0.096000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66627,857110,6282,Q25619.1,"9606,9606",2,0,Q25619.1,6282,Onchocerca volvulus,29,SMSNIPEEYKEFIPE,1,2.973557,81,149,17,42,1,2065.20,0.133333,0.3

In [ ]:
# Importing the dataset
holdout_ovolvulus = pd.read_csv('./input/02_holdout.csv')

holdout_ovolvulus

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Info_epitope_id,Info_sourceOrg_id,Info_protein_id,Info_host_id,Info_n_Positive,Info_n_Negative,Info_TSeq_accver,Info_TSeq_taxid,Info_TSeq_orgname,Info_center_pos,Info_window_seq,Class,feat_seq_entropy,feat_C_atoms,feat_H_atoms,feat_N_atoms,feat_O_atoms,feat_S_atoms,feat_molecular_weight,feat_Perc_Tiny,feat_Perc_Small,feat_Perc_Aliphatic,feat_Perc_Aromatic,feat_Perc_NonPolar,feat_Perc_Polar,feat_Perc_Charged,feat_Perc_Basic,feat_Perc_Acidic,feat_PP1,feat_PP2,feat_PP3,feat_KF1,feat_KF2,feat_KF3,feat_KF4,feat_KF5,feat_KF6,feat_KF7,feat_KF8,feat_KF9,...,feat_Perc_WA,feat_Perc_WC,feat_Perc_WD,feat_Perc_WE,feat_Perc_WF,feat_Perc_WG,feat_Perc_WH,feat_Perc_WI,feat_Perc_WK,feat_Perc_WL,feat_Perc_WM,feat_Perc_WN,feat_Perc_WP,feat_Perc_WQ,feat_Perc_WR,feat_Perc_WS,feat_Perc_WT,feat_Perc_WV,feat_Perc_WW,feat_Perc_WY,feat_Perc_YA,feat_Perc_YC,feat_Perc_YD,feat_Perc_YE,feat_Perc_YF,feat_Perc_YG,feat_Perc_YH,feat_Perc_YI,feat_Perc_YK,feat_Perc_YL,feat_Perc_YM,feat_Perc_YN,feat_Perc_YP,feat_Perc_YQ,feat_Perc_YR,feat_Perc_YS,feat_Perc_YT,feat_Perc_YV,feat_Perc_YW,feat_Perc_YY
0,851085,6282,A0A044QU24,"9606,9606",1,1,NaN,6282,Onchocerca volvulus,67,KMQGKANAISILPPT,1,3.373557,69,149,19,34,1,1821.09,0.333333,0.533333,0.333333,0.000000,0.600000,0.400000,0.133333,0.133333,0.000000,-0.294000,-0.366000,0.191333,-0.025333,-0.404667,-0.046000,0.052667,0.294667,-0.369333,0.165333,-0.264667,-0.045333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,851085,6282,A0A044QU24,"9606,9606",1,1,NaN,6282,Onchocerca volvulus,68,MQGKANAISILPPTV,1,3.506891,68,146,18,34,1,1792.05,0.333333,0.600000,0.400000,0.000000,0.666667,0.333333,0.066667,0.066667,0.000000,-0.400667,-0.401333,0.115333,-0.052000,-0.506667,0.105333,-0.087333,0.225333,-0.315333,0.017333,-0.255333,-0.044000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,851085,6282,A0A044QU24,"9606,9606",1,1,NaN,6282,Onchocerca volvulus,69,QGKANAISILPPTVA,1,3.323231,66,142,18,34,0,1731.93,0.400000,0.666667,0.466667,0.000000,0.666667,0.333333,0.066667,0.066667,0.000000,-0.410000,-0.454000,0.141333,-0.062667,-0.630000,0.068667,-0.056667,0.030000,-0.468667,-0.013333,-0.268000,0.054667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,851085,6282,A0A044QU24,"9606,9606",1,1,NaN,6282,Onchocerca volvulus,70,GKANAISILPPTVAA,1,3.106891,64,139,17,33,0,1674.87,0.466667,0.733333,0.533333,0.000000,0.733333,0.266667,0.066667,0.066667,0.000000,-0.526000,-0.484667,0.187333,-0.135333,-0.757333,-0.000667,-0.148000,-0.105333,-0.560000,-0.082667,-0.226000,0.070667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,851085,6282,A0A044QU24,"9606,9606",1,1,NaN,6282,Onchocerca volvulus,71,KANAISILPPTVAAG,1,3.106891,64,139,17,33,0,1674.87,0.466667,0.733333,0.533333,0.000000,0.733333,0.266667,0.066667,0.066667,0.000000,-0.526000,-0.484667,0.187333,-0.135333,-0.757333,-0.000667,-0.148000,-0.105333,-0.560000,-0.082667,-0.226000,0.070667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22205,857570,6282,Q9NJD8.1,"9606,9606",0,2,Q9NJD8.1,6282,Onchocerca volvulus,331,AKKLKELSKRATFVR,-1,2.973557,80,171,25,34,0,2027.36,0.266667,0.333333,0.333333,0.066667,0.4

In [ ]:
train_ovolvulus.Class.value_counts()

 1    11532
-1    10678
Name: Class, dtype: int64

In [ ]:
feature_columns = [col for col in train_ovolvulus if col.startswith('feat_')]
target = "Class"

In [ ]:
!pip install yellowbrick==0.9.1 scikit-learn==0.22.2
!pip install imbalanced-learn
!pip install mlrose
!pip install imblearn

import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

In [ ]:
#Defining the pipeline
from sklearn.model_selection import GroupKFold
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.preprocessing import MinMaxScaler


pipeline = imbpipeline(steps = [['smote', SMOTE()],
                                ['scaler', MinMaxScaler()],
                                ['classifier', RandomForestClassifier()]])


In [ ]:
X_train = train_ovolvulus[feature_columns] 
y_train = train_ovolvulus[target] 
X_test = holdout_ovolvulus[feature_columns]  
y_test = holdout_ovolvulus[target] 

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
from sklearn.metrics import fbeta_score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

model = pipeline;

model.fit(X_train, y_train)

preds = model.predict(X_test)

print("ROC_AUC:", roc_auc_score(y_test, preds))

print(classification_report(y_test, preds))


ROC_AUC: 0.8254319746897434
              precision    recall  f1-score   support

          -1       0.82      0.82      0.82     10678
           1       0.83      0.83      0.83     11532

    accuracy                           0.83     22210
   macro avg       0.83      0.83      0.83     22210
weighted avg       0.83      0.83      0.83     22210

